In [1]:
from sklearn.ensemble import RandomForestRegressor
# from sklearn.feature_selection import SelectFromModel

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./Output/NEES_summary_with_heliquest_Median_Transformed.csv')

In [4]:
df.columns

Index(['AA_seq', 'AH', 'Median', 'Mean', 'NEES_binned', 'phil_A', 'phob_A',
       'phil_G', 'phob_G', 'phil_V', 'phob_V', 'phil_L', 'phob_L', 'phil_I',
       'phob_I', 'phil_F', 'phob_F', 'phil_W', 'phob_W', 'phil_M', 'phob_M',
       'phil_Y', 'phob_Y', 'phil_C', 'phob_C', 'phil_S', 'phob_S', 'phil_T',
       'phob_T', 'phil_R', 'phob_R', 'phil_K', 'phob_K', 'phil_N', 'phob_N',
       'phil_Q', 'phob_Q', 'phil_D', 'phob_D', 'phil_E', 'phob_E', 'phil_H',
       'phob_H', 'phil_P', 'phob_P', 'Length', 'Hydrophobicity', 'H_moment',
       'Netcharge', 'Dfactor', 'phil_S_T', 'phil_R_K', 'phil_L_I', 'phil_F_W',
       'phil_M_C', 'phil_D_E', 'phil_N_Q', 'phob_S_T', 'phob_R_K', 'phob_L_I',
       'phob_F_W', 'phob_M_C', 'phob_D_E', 'phob_N_Q', 'Median_Log',
       'Median_boxcox', 'Median_boxcox_No1shift'],
      dtype='object')

In [ ]:
# Both faces
x_columns = ['phil_S_T', 'phil_R_K', 'phil_L_I', 'phil_F_W', 'phil_M_C', 'phil_D_E', 'phil_N_Q', 
                       'phil_A', 'phil_G', 'phil_V', 'phil_Y', 'phil_H', 'phil_P'] + ['phob_S_T', 'phob_R_K', 'phob_L_I', 'phob_F_W', 'phob_M_C', 'phob_D_E', 'phob_N_Q', 
                       'phob_A', 'phob_G', 'phob_V', 'phob_Y', 'phob_H', 'phob_P']

# Prepare features and target variable for training the Random Forest Classifier
X_train = df[x_columns]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['NEES_binned'])

In [ ]:
# Train a Random Forest Regressor with the complete set of predictors and Box-Cox transformed target variable
rf_regressor_complete = RandomForestRegressor(n_estimators=100, random_state=0)
rf_regressor_complete.fit(X_complete, y_boxcox)

# Get feature importances from the regressor
importances_complete = rf_regressor_complete.feature_importances_
feature_importance_complete = pd.Series(importances_complete, index=complete_predictors).sort_values(ascending=False)

# Display the importances
feature_importance_complete